In [20]:
import json
import pandas as pd
import xlsxwriter
import glob
import os



#Initialize

# 内含每种角色：

roles = {
    '狼人总数':{'局数':0,'胜局数':0,'上警数':0,'警徽天数':0,'警徽投票数':0,'警徽正确票数':0,'警徽错误票数':0,'警徽弃票数':0,
            '放逐投票数':0,'放逐正确票数':0,'放逐错误票数':0,'放逐弃票数':0,
            '猎杀数':0,'猎杀神':0,'猎杀民':0,'猎杀狼':0,'自爆数':0},
    '好人总数':{'局数':0,'胜局数':0,'上警数':0,'警徽天数':0,'警徽投票数':0,'警徽正确票数':0,'警徽错误票数':0,'警徽弃票数':0,
           '放逐投票数':0,'放逐正确票数':0,'放逐错误票数':0,'放逐弃票数':0}}

times = {
    '第一夜':0.5,'第一天':1,'第二夜':1.5,'第二天':2,'第三夜':2.5,'第三天':3,'第四夜':3.5,'第四天':4,
    '第五夜':4.5,'第五天':5,'第六夜':5.5,'第六天':6,'第七夜':6.5,'第七天':7, '警上': 1
}
nights = {
    '第一夜':0.5,'第二夜':1.5,'第三夜':2.5,'第四夜':3.5,
    '第五夜':4.5,'第六夜':5.5,'第七夜':6.5, '警上': 1
}

players = {}

god_list = ['预言家','女巫','猎人','白痴','守卫','乌鸦','魔术师','骑士','九尾妖狐','禁言长老','潜行者','狐狸','熊',
           '操纵月亮的女孩','小红帽','摄梦人','阴阳使者','守墓人','通灵师','牧师','黑市商人','猎魔人','武僧','小女孩',
            '占卜师','蛊惑师','公爵','锈剑骑士']
villager_list = ['平民','混血儿','野孩子']

In [21]:
path_to_json = '/'
files = [f for f in os.listdir('.') if os.path.isfile(f)]
json_files = [json_file for json_file in files if json_file.endswith('.json')]
for json_file in json_files:
    with open(json_file) as f:
        data = json.load(f)

        # 游戏时长
        game_time = 0
        for i in range(1,13):
            seat = data['data'][str(i)]
            if 'actions' in seat.keys():
                for night in seat['actions']:
                    game_time = max(game_time, times[night])
            if 'death_round' in seat.keys():
                game_time = max(game_time, times[seat['death_round']])



        # 给每位玩家创造数据
        for i in range(1,13):
            seat = data['data'][str(i)]
            player = seat['name']
            if player not in players:
                players[player] = {
        '总数':{'局数':0,'胜局数':0,'上警数':0,'警徽天数':0,'存活局数':0,
              '放逐局数':0,'被神杀局数':0,'被首验数':0,'被查验数':0,'被查验轮次':0,'被首刀数':0,'得分数':0},
        '狼人':{'局数':0,'胜局数':0,'警徽投票数':0,'警徽正确票数':0,'警徽错误票数':0,'警徽弃票数':0,'上警数':0,'警徽天数':0,
              '放逐投票数':0,'放逐正确票数':0,'放逐错误票数':0,'放逐弃票数':0,'猎杀数':0,'猎杀神':0,
              '猎杀民':0,'自刀':0,'寿命天数':0,'存活局数':0,'放逐局数':0,'被神杀局数':0,'被刀杀局数':0,'自爆局数':0,'被查验数':0,'被查验轮次':0},
        '神':{'局数':0,'胜局数':0,'警徽投票数':0,'警徽正确票数':0,'警徽错误票数':0,'警徽弃票数':0,'上警数':0,'警徽天数':0,'被查验数':0,'被查验轮次':0,
             '放逐投票数':0,'放逐正确票数':0,'放逐错误票数':0,'放逐弃票数':0,'寿命天数':0,'存活局数':0,'放逐局数':0,'被刀杀局数':0,'被神杀局数':0},
        '平民':{'局数':0,'胜局数':0,'警徽投票数':0,'警徽正确票数':0,'警徽错误票数':0,'警徽弃票数':0,'上警数':0,'警徽天数':0,'被查验数':0,'被查验轮次':0,
              '放逐投票数':0,'放逐错误票数':0,'放逐正确票数':0,'放逐弃票数':0,'寿命天数':0,'存活局数':0,'放逐局数':0,'被刀杀局数':0,'被神杀局数':0}}

        # 总数信息
        # 局数信息
        roles['狼人总数']['局数'] += 1
        roles['狼人总数']['胜局数'] += (data['winner'] == '狼人')
        roles['好人总数']['局数'] += 1
        roles['好人总数']['胜局数'] += (data['winner'] == '好人')

        badge_comp_wolf = 0
        badge_comp_good = 0
        wolf_kill_dict = {}
        guard_dict = {}


        # 内含每位玩家：
        for i in range(1,13):
            seat = data['data'][str(i)]
            player = seat['name']
            role = seat['role']
            camp = seat['final_camp']
            if camp == '好人':
                if role in ['平民','混血儿','野孩子']:
                    group = '平民'
                else:
                    group = '神'
            else:
                group = '狼人'
            votes = seat['votes']
            result = seat['result']


    #'总数':{被首验数':0,'得分数':0},

            # 给每种角色创造数据
            if role not in roles.keys():
                roles[role] = {'局数':0,'胜局数':0,'上警数':0,'警徽投票数':0,'警徽正确票数':0,'警徽错误票数':0,'警徽弃票数':0,
               '放逐投票数':0,'放逐正确票数':0,'放逐错误票数':0,'放逐弃票数':0}

            # 局数信息 
            players[player]['总数']['局数'] += 1
            players[player][group]['局数'] += 1
            roles[role]['局数'] += 1

            players[player]['总数']['胜局数'] += (result == '胜利')
            players[player][group]['胜局数'] += (result == '胜利')
            roles[role]['胜局数'] += (result == '胜利')

            # 上警信息
            if votes['警徽'] == '上警':
                players[player]['总数']['上警数'] += 1
                players[player][group]['上警数'] += 1
                roles[role]['上警数'] += 1
                if group == '狼人':
                    badge_comp_wolf += 1
                else:
                    badge_comp_good += 1

            # 拿警徽信息
            if 'badge' in seat.keys():
                if 'death_round' in seat.keys():
                    badge_time = (times[seat['death_round']] - times[seat['badge']])
                else:
                    badge_time = (game_time - times[seat['badge']])
                players[player]['总数']['警徽天数'] += badge_time + 1
                players[player][group]['警徽天数'] += badge_time + 1
                if group == '狼人':
                    roles['狼人总数']['警徽天数'] += badge_time + 1
                else:
                    roles['好人总数']['警徽天数'] += badge_time + 1

            # 死亡和存活信息                                                    
            if 'death_method' in seat.keys():
                players[player][group]['寿命天数'] += times[seat['death_round']]
                if seat['death_method'] == '放逐':
                    players[player]['总数']['放逐局数'] += 1
                    players[player][group]['放逐局数'] += 1
                elif seat['death_method'] in ['毒杀','枪杀']:
                    players[player]['总数']['被神杀局数'] += 1
                    players[player][group]['被神杀局数'] += 1
                elif seat['death_method'] == '自爆':
                    players[player]['狼人']['自爆局数'] += 1
                    roles['狼人总数']['自爆数'] += 1
                elif seat['death_method'] == '猎杀':
                    players[player][group]['被刀杀局数'] += 1
            else:
                players[player]['总数']['存活局数'] += 1
                players[player][group]['寿命天数'] += game_time
                players[player][group]['存活局数'] += 1


            # 投票信息
            for key in votes:
                # 警徽投票
                if key in ['警徽','警徽 pk']:
                    if votes[key] not in ['上警','-','禁票']:
                        players[player][group]['警徽投票数'] += 1
                        roles[role]['警徽投票数'] += 1
                        if group == '狼人':
                            roles['狼人总数']['警徽投票数'] += 1
                        else:
                            roles['好人总数']['警徽投票数'] += 1
                        if votes[key] == '弃票':
                            roles[role]['警徽弃票数'] += 1
                            if group == '狼人':
                                players[player][group]['警徽弃票数'] += 1
                                roles['狼人总数']['警徽弃票数'] += 1
                            else:
                                players[player][group]['警徽弃票数'] += 1
                                roles['好人总数']['警徽弃票数'] += 1
                        else:
                            if group == '狼人':
                                players[player]['狼人']['警徽正确票数'] += (data['data'][votes[key]]['initial_camp'] == '狼人')
                                players[player]['狼人']['警徽错误票数'] += (data['data'][votes[key]]['initial_camp'] != '狼人')
                                roles['狼人总数']['警徽正确票数'] += (data['data'][votes[key]]['initial_camp'] == '狼人')
                                roles['狼人总数']['警徽错误票数'] += (data['data'][votes[key]]['initial_camp'] != '狼人')
                                roles[role]['警徽正确票数'] += (data['data'][votes[key]]['initial_camp'] == '狼人')
                                roles[role]['警徽错误票数'] += (data['data'][votes[key]]['initial_camp'] != '狼人')
                            else:
                                players[player][group]['警徽正确票数'] += (data['data'][votes[key]]['initial_camp'] != '狼人')
                                players[player][group]['警徽错误票数'] += (data['data'][votes[key]]['initial_camp'] == '狼人')
                                roles['好人总数']['警徽正确票数'] += (data['data'][votes[key]]['initial_camp'] != '狼人')
                                roles['好人总数']['警徽错误票数'] += (data['data'][votes[key]]['initial_camp'] == '狼人')
                                roles[role]['警徽正确票数'] += (data['data'][votes[key]]['initial_camp'] != '狼人')
                                roles[role]['警徽错误票数'] += (data['data'][votes[key]]['initial_camp'] == '狼人')



                # 放逐投票
                else:
                    if votes[key] not in ['-','禁票']:
                        players[player][group]['放逐投票数'] += 1
                        roles[role]['放逐投票数'] += 1
                        if group == '狼人':
                            roles['狼人总数']['放逐投票数'] += 1
                        else:
                            roles['好人总数']['放逐投票数'] += 1
                        if votes[key] == '弃票':
                            roles[role]['放逐弃票数'] += 1
                            if group == '狼人':
                                players[player][group]['放逐弃票数'] += 1
                                roles['狼人总数']['放逐弃票数'] += 1
                            else:
                                players[player][group]['放逐弃票数'] += 1
                                roles['好人总数']['放逐弃票数'] += 1
                        else:
                            if group == '狼人':
                                players[player]['狼人']['放逐正确票数'] += (data['data'][votes[key]]['initial_camp'] != '狼人')
                                players[player]['狼人']['放逐错误票数'] += (data['data'][votes[key]]['initial_camp'] == '狼人')
                                roles['狼人总数']['放逐正确票数'] += (data['data'][votes[key]]['initial_camp'] != '狼人')
                                roles['狼人总数']['放逐错误票数'] += (data['data'][votes[key]]['initial_camp'] == '狼人')
                                roles[role]['放逐正确票数'] += (data['data'][votes[key]]['initial_camp'] != '狼人')
                                roles[role]['放逐错误票数'] += (data['data'][votes[key]]['initial_camp'] == '狼人')
                            else:
                                players[player][group]['放逐正确票数'] += (data['data'][votes[key]]['initial_camp'] == '狼人')
                                players[player][group]['放逐错误票数'] += (data['data'][votes[key]]['initial_camp'] != '狼人')
                                roles['好人总数']['放逐正确票数'] += (data['data'][votes[key]]['initial_camp'] == '狼人')
                                roles['好人总数']['放逐错误票数'] += (data['data'][votes[key]]['initial_camp'] != '狼人')
                                roles[role]['放逐正确票数'] += (data['data'][votes[key]]['initial_camp'] == '狼人')
                                roles[role]['放逐错误票数'] += (data['data'][votes[key]]['initial_camp'] != '狼人')



            # 狼人猎杀信息
            if group == '狼人':
                if 'actions' in seat.keys():    
                    actions = seat['actions']
                    for night in actions:
                        if night not in nights.keys():
                            continue
                        for action in actions[night]:
                            if action['ability'] == '猎杀':
                                kill_target_seat = str(action['target_seat'])
                                players[player]['狼人']['猎杀数'] += 1
                                players[player]['狼人']['猎杀民'] += (action['target_role'] in villager_list)
                                players[player]['狼人']['猎杀神'] += (action['target_role'] in god_list)
                                players[player]['狼人']['自刀'] += (kill_target_seat == seat)
                                wolf_kill_dict[night] = kill_target_seat

            # 预言家信息    
            if role == '预言家':
                # Initialize
                if '查验数' not in roles[role].keys():
                    roles['预言家']['查验数']=0
                    roles['预言家']['查验民']=0
                    roles['预言家']['查验神']=0
                    roles['预言家']['查验狼']=0
                    roles['预言家']['拿到警徽数']=0
                    roles['预言家']['首天被放逐局数']=0
                    roles['预言家']['未被放逐存活天数']=0

                if 'badge' in seat.keys():
                    roles['预言家']['拿到警徽数'] += 1
                if ('death_method' in seat.keys() and seat['death_method'] == '放逐' and seat['death_round'] == '第一天'):
                    roles['预言家']['首天被放逐局数'] += 1
                else:
                    if 'death_method' in seat.keys():
                        roles['预言家']['未被放逐存活天数'] += times[seat['death_round']]
                    else:
                        roles['预言家']['未被放逐存活天数'] += game_time

                actions = seat['actions']
                for night in actions:
                    if night == '第一夜':
                        for action in actions[night]:
                            if action['ability'] == '查验':
                                target_player = data['data'][str(action['target_seat'])]['name']
                                players[target_player]['总数']['被首验数'] += 1
                    for action in actions[night]:
                        if action['ability'] == '查验':
                            roles['预言家']['查验数'] += 1
                            check_target_seat = str(action['target_seat'])
                            roles['预言家']['查验民'] += (action['target_role'] in villager_list)
                            roles['预言家']['查验神'] += (action['target_role'] in god_list)
                            roles['预言家']['查验狼'] += (action['check_result'] == '狼人')
                            target_player = data['data'][str(action['target_seat'])]['name']
                            players[target_player]['总数']['被查验数'] += 1
                            players[target_player][group]['被查验数'] += 1
                            players[target_player]['总数']['被查验轮次'] += (times[night] + 0.5)
                            players[target_player][group]['被查验轮次'] += (times[night] + 0.5)

            # 女巫信息
            if role == '女巫':
                # Initialize
                if '毒杀数' not in roles[role].keys():
                    roles['女巫']['首夜中刀数']=0
                    roles['女巫']['首夜未中刀解药数']=0
                    roles['女巫']['毒杀数']=0
                    roles['女巫']['毒杀狼']=0
                    roles['女巫']['毒杀神']=0
                    roles['女巫']['毒杀民']=0
                actions = seat['actions']
                if ('death_round' in seat.keys() and seat['death_round'] == '第一夜'):
                    roles['女巫']['首夜中刀数'] += 1
                for night in actions:
                    if night == '第一夜':
                        for action in actions[night]:
                            if action['ability'] == '解救':
                                roles['女巫']['首夜未中刀解药数'] += 1
                    for action in actions[night]:
                        if action['ability'] == '毒杀':
                            roles['女巫']['毒杀数'] += 1
                            poison_target_seat = str(action['target_seat'])
                            roles['女巫']['毒杀民'] += (action['target_role'] in villager_list)
                            roles['女巫']['毒杀神'] += (action['target_role'] in god_list)
                            roles['女巫']['毒杀狼'] += (data['data'][poison_target_seat]['final_camp'] == '狼人')

            # 猎人信息
            if role == '猎人':
                # Initialize
                if '毒杀数' not in roles[role].keys():
                    roles['猎人']['枪杀数']=0
                    roles['猎人']['枪杀狼']=0
                    roles['猎人']['枪杀神']=0
                    roles['猎人']['枪杀民']=0
                if 'actions' in seat.keys():
                    actions = seat['actions']
                    for night in actions:
                        if actions[night]['ability'] == '枪杀':
                            roles['猎人']['枪杀数'] += 1
                            shoot_target_seat = str(action['target_seat'])
                            roles['猎人']['枪杀民'] += (action['target_role'] in villager_list)
                            roles['猎人']['枪杀神'] += (action['target_role'] in god_list)
                            roles['猎人']['枪杀狼'] += (data['data'][shoot_target_seat]['final_camp'] == '狼人')


       # '骑士':{'决斗数':0,'正确决斗数':0},
       # '白狼王':{'自爆数':0,'警上自爆数':0,'警下自爆数':0,'自爆带神数':0}

            # 白痴信息
            if role == '白痴':
                # Initialize
                if '放逐数' not in roles[role].keys():
                    roles['白痴']['放逐数']=0
                if ('death_method' in seat.keys() and seat['death_method'] == '放逐'):
                    roles['白痴']['放逐数'] += 1

            # 守卫信息
            if role == '守卫':
                # Initialize
                if '守护数' not in roles[role].keys():
                    roles['守卫']['守护数']=0
                    roles['守卫']['正确守护数']=0
                actions = seat['actions']
                for night in actions:
                    for action in actions[night]:
                        if action['ability'] == '守护':
                            roles['守卫']['守护数'] += 1
                            guard_target_seat = str(action['target_seat'])
                            guard_dict[night] = guard_target_seat




        # 最后更新总数信息
        roles['狼人总数']['上警数'] += badge_comp_wolf
        roles['好人总数']['上警数'] += badge_comp_good

        first_kill_target = wolf_kill_dict['第一夜']
        target_player = data['data'][str(first_kill_target)]['name']
        players[target_player]['总数']['被首刀数'] += 1

        for night in wolf_kill_dict:
            seat = wolf_kill_dict[night]
            roles['狼人总数']['猎杀数'] += 1
            if data['data'][seat]['role'] in god_list:
                roles['狼人总数']['猎杀神'] += 1
            elif data['data'][seat]['role'] in villager_list:
                roles['狼人总数']['猎杀民'] += 1
            elif data['data'][seat]['initial_camp'] == '狼人':
                roles['狼人总数']['猎杀狼'] += 1
                
        for night in guard_dict:
            seat = guard_dict[night]
            if night in wolf_kill_dict and wolf_kill_dict[night] == seat:
                roles['守卫']['正确守护数'] += 1

In [22]:
# 算出平均数



  #  '狼人':{'局数':0''自爆局数':0},

for name in players:
    player = players[name]
    for key in player.keys():
        player[key]['胜率'] = 'NA' if player[key]['局数']==0 else player[key]['胜局数'] / player[key]['局数']
        player[key]['上警率'] = 'NA' if player[key]['局数']==0 else player[key]['上警数'] / player[key]['局数']
        player[key]['平均警徽天数'] = 'NA' if player[key]['局数']==0 else player[key]['警徽天数'] / player[key]['局数']
        player[key]['存活率'] = 'NA' if player[key]['局数']==0 else player[key]['存活局数'] / player[key]['局数']
        player[key]['放逐率'] = 'NA' if player[key]['局数']==0 else player[key]['放逐局数'] / player[key]['局数']
        player[key]['被神杀率'] = 'NA' if player[key]['局数']==0 else player[key]['被神杀局数'] / player[key]['局数']
        player[key]['被查验率'] = 'NA' if player[key]['局数']==0 else player[key]['被查验数'] / player[key]['局数']
        player[key]['平均被查验轮次'] = 'NA' if player[key]['被查验数']==0 else player[key]['被查验轮次'] / player[key]['被查验数']
        if key == '总数':
            player[key]['被首验率'] = 'NA' if player[key]['局数']==0 else player[key]['被首验数'] / player[key]['局数']
            player[key]['被首刀率'] = 'NA' if player[key]['局数']==0 else player[key]['被首刀数'] / player[key]['局数']
        elif key == '狼人':
            player[key]['平均寿命天数'] = 'NA' if player[key]['局数']==0 else player[key]['寿命天数'] / player[key]['局数']
            player[key]['警徽投票正确率'] = 'NA' if player[key]['警徽投票数']==0 else player[key]['警徽正确票数'] / player[key]['警徽投票数']
            player[key]['警徽投票错误率'] = 'NA' if player[key]['警徽投票数']==0 else player[key]['警徽错误票数'] / player[key]['警徽投票数']
            player[key]['警徽投票弃票率'] = 'NA' if player[key]['警徽投票数']==0 else player[key]['警徽弃票数'] / player[key]['警徽投票数']
            player[key]['放逐投票正确率'] = 'NA' if player[key]['放逐投票数']==0 else player[key]['放逐正确票数'] / player[key]['放逐投票数']
            player[key]['放逐投票错误率'] = 'NA' if player[key]['放逐投票数']==0 else player[key]['放逐错误票数'] / player[key]['放逐投票数']
            player[key]['放逐投票弃票率'] = 'NA' if player[key]['放逐投票数']==0 else player[key]['放逐弃票数'] / player[key]['放逐投票数']
            player[key]['刀中神率'] = 'NA' if player[key]['猎杀数']==0 else player[key]['猎杀神'] / player[key]['猎杀数']
            player[key]['刀中民率'] = 'NA' if player[key]['猎杀数']==0 else player[key]['猎杀民'] / player[key]['猎杀数']
            player[key]['自爆率'] = 'NA' if player[key]['局数']==0 else player[key]['自爆局数'] / player[key]['局数']
        else:
            player[key]['平均寿命天数'] = 'NA' if player[key]['局数']==0 else player[key]['寿命天数'] / player[key]['局数']
            player[key]['被刀杀率'] = 'NA' if player[key]['局数']==0 else player[key]['被刀杀局数'] / player[key]['局数']
            player[key]['警徽投票正确率'] = 'NA' if player[key]['警徽投票数']==0 else player[key]['警徽正确票数'] / player[key]['警徽投票数']
            player[key]['警徽投票错误率'] = 'NA' if player[key]['警徽投票数']==0 else player[key]['警徽错误票数'] / player[key]['警徽投票数']
            player[key]['警徽投票弃票率'] = 'NA' if player[key]['警徽投票数']==0 else player[key]['警徽弃票数'] / player[key]['警徽投票数']
            player[key]['放逐投票正确率'] = 'NA' if player[key]['放逐投票数']==0 else player[key]['放逐正确票数'] / player[key]['放逐投票数']
            player[key]['放逐投票错误率'] = 'NA' if player[key]['放逐投票数']==0 else player[key]['放逐错误票数'] / player[key]['放逐投票数']
            player[key]['放逐投票弃票率'] = 'NA' if player[key]['放逐投票数']==0 else player[key]['放逐弃票数'] / player[key]['放逐投票数']
            
            
for role_name in roles:
    role = roles[role_name]
    if role_name == '狼人总数':
        role['胜率'] = 'NA' if role['局数']==0 else role['胜局数'] / role['局数']
        role['平均上警人数'] = 'NA' if role['局数']==0 else role['上警数'] / role['局数']
        role['平均警徽天数'] = 'NA' if role['局数']==0 else role['警徽天数'] / role['局数']
        role['警徽投票正确率'] = 'NA' if role['警徽投票数']==0 else role['警徽正确票数'] / role['警徽投票数']
        role['警徽投票错误率'] = 'NA' if role['警徽投票数']==0 else role['警徽错误票数'] / role['警徽投票数']
        role['警徽弃票率'] = 'NA' if role['警徽投票数']==0 else role['警徽弃票数'] / role['警徽投票数']
        role['放逐投票正确率'] = 'NA' if role['放逐投票数']==0 else role['放逐正确票数'] / role['放逐投票数']
        role['放逐投票错误率'] = 'NA' if role['放逐投票数']==0 else role['放逐错误票数'] / role['放逐投票数']
        role['放逐弃票率'] = 'NA' if role['放逐投票数']==0 else role['放逐弃票数'] / role['放逐投票数']
        role['刀神率'] = 'NA' if role['猎杀数']==0 else role['猎杀神'] / role['猎杀数']
        role['刀民率'] = 'NA' if role['猎杀数']==0 else role['猎杀民'] / role['猎杀数']
        role['刀狼率'] = 'NA' if role['猎杀数']==0 else role['猎杀狼'] / role['猎杀数']
        role['平均自爆人数'] = 'NA' if role['局数']==0 else role['自爆数'] / role['局数']
    elif role_name == '好人总数':
        role['胜率'] = 'NA' if role['局数']==0 else role['胜局数'] / role['局数']
        role['平均上警人数'] = 'NA' if role['局数']==0 else role['上警数'] / role['局数']
        role['平均警徽天数'] = 'NA' if role['局数']==0 else role['警徽天数'] / role['局数']
        role['警徽投票正确率'] = 'NA' if role['警徽投票数']==0 else role['警徽正确票数'] / role['警徽投票数']
        role['警徽投票错误率'] = 'NA' if role['警徽投票数']==0 else role['警徽错误票数'] / role['警徽投票数']
        role['警徽弃票率'] = 'NA' if role['警徽投票数']==0 else role['警徽弃票数'] / role['警徽投票数']
        role['放逐投票正确率'] = 'NA' if role['放逐投票数']==0 else role['放逐正确票数'] / role['放逐投票数']
        role['放逐投票错误率'] = 'NA' if role['放逐投票数']==0 else role['放逐错误票数'] / role['放逐投票数']
        role['放逐弃票率'] = 'NA' if role['放逐投票数']==0 else role['放逐弃票数'] / role['放逐投票数']
    else:
        role['胜率'] = 'NA' if role['局数']==0 else role['胜局数'] / role['局数']
        role['上警率'] = 'NA' if role['局数']==0 else role['上警数'] / role['局数']
        role['警徽投票正确率'] = 'NA' if role['警徽投票数']==0 else role['警徽正确票数'] / role['警徽投票数']
        role['警徽投票错误率'] = 'NA' if role['警徽投票数']==0 else role['警徽错误票数'] / role['警徽投票数']
        role['警徽弃票率'] = 'NA' if role['警徽投票数']==0 else role['警徽弃票数'] / role['警徽投票数']
        role['放逐投票正确率'] = 'NA' if role['放逐投票数']==0 else role['放逐正确票数'] / role['放逐投票数']
        role['放逐投票错误率'] = 'NA' if role['放逐投票数']==0 else role['放逐错误票数'] / role['放逐投票数']
        role['放逐弃票率'] = 'NA' if role['放逐投票数']==0 else role['放逐弃票数'] / role['放逐投票数']
        if role_name == '预言家':
            role['验神率'] = 'NA' if role['局数']==0 else role['查验神'] / role['局数']
            role['验民率'] = 'NA' if role['局数']==0 else role['查验民'] / role['局数']
            role['验狼率'] = 'NA' if role['局数']==0 else role['查验狼'] / role['局数']
            role['警徽率'] = 'NA' if role['局数']==0 else role['拿到警徽数'] / role['局数']
            role['首天放逐率'] = 'NA' if role['局数']==0 else role['首天被放逐局数'] / role['局数']
            role['未被放逐存活平均天数'] = 'NA' if (role['局数']-role['首天被放逐局数'])==0 else role['未被放逐存活天数'] / (role['局数']-role['首天被放逐局数'])
        elif role_name == '女巫':
            role['首夜中刀率'] = 'NA' if role['局数']==0 else role['首夜中刀数'] / role['局数']
            role['首夜未中刀解药率'] = 'NA' if (role['局数']-role['首夜中刀数'])==0 else role['首夜未中刀解药数'] / (role['局数']-role['首夜中刀数'])
            role['毒神率'] = 'NA' if role['毒杀数']==0 else role['毒杀神'] / role['毒杀数']
            role['毒民率'] = 'NA' if role['毒杀数']==0 else role['毒杀民'] / role['毒杀数']
            role['毒狼率'] = 'NA' if role['毒杀数']==0 else role['毒杀狼'] / role['毒杀数']
        elif role_name == '猎人':
            role['枪神率'] = 'NA' if role['枪杀数']==0 else role['枪杀神'] / role['枪杀数']
            role['枪民率'] = 'NA' if role['枪杀数']==0 else role['枪杀民'] / role['枪杀数']
            role['枪狼率'] = 'NA' if role['枪杀数']==0 else role['枪杀狼'] / role['枪杀数']
        elif role_name == '守卫':
            role['守护成功率'] = 'NA' if role['守护数']==0 else role['正确守护数'] / role['守护数']
        elif role_name == '白痴':
            role['放逐率'] = 'NA' if role['局数']==0 else role['放逐数'] / role['局数']
            


In [23]:
# Transport json to pandas
rowindices = set([])
for role_name in roles.keys():
    rowindices = rowindices.union(set([key for key in roles[role_name]]))
rowindices = sorted(list(rowindices))
rolesdf = pd.DataFrame(index = roles.keys(),columns = rowindices)

for role_name in roles.keys():
    for key in roles[role_name]:
        rolesdf.loc[role_name][key] = roles[role_name][key]

player_list = list(players.keys())
player = players['胡雨章']
alldf = pd.DataFrame(index = player_list,columns = player['总数'].keys())
wolfdf = pd.DataFrame(index = player_list,columns = player['狼人'].keys())
villagerdf = pd.DataFrame(index = player_list,columns = player['平民'].keys())
goddf = pd.DataFrame(index = player_list,columns = player['神'].keys())

for player_name in players:
    player = players[player_name]
    for key in player['总数'].keys():
        alldf.loc[player_name][key] = player['总数'][key]
    for key in player['狼人'].keys():
        wolfdf.loc[player_name][key] = player['狼人'][key]
    for key in player['平民'].keys():
        villagerdf.loc[player_name][key] = player['平民'][key]
    for key in player['神'].keys():
        goddf.loc[player_name][key] = player['神'][key]
        

In [24]:
# export pandas to excel
writer = pd.ExcelWriter('roles.xlsx', engine='xlsxwriter')
rolesdf.to_excel(writer, sheet_name='sheet')
writer.save()


writer = pd.ExcelWriter('players.xlsx', engine='xlsxwriter')
alldf.to_excel(writer, sheet_name='总数')
wolfdf.to_excel(writer, sheet_name='狼人')
goddf.to_excel(writer, sheet_name='神')
villagerdf.to_excel(writer, sheet_name='平民')

writer.save()